In [11]:
from rdflib import URIRef, BNode, Literal
from rdflib.namespace import RDF, RDFS
from pyfuseki import FusekiUpdate, FusekiQuery

In [14]:
acervoUpdate = FusekiUpdate('http://localhost:3030', 'acervo')
acervoQuery = FusekiQuery('http://localhost:3030', 'acervo')

In [18]:
sparql_str = "DESCRIBE <https://bibliokeia.com/resources/work/bk-2>"
response = acervoQuery.run_sparql(sparql_str)
response


c:\Users\Usuário\Desktop\BiblioKeia\venv\lib\site-packages\SPARQLWrapper\Wrapper.py:794: RuntimeWarning: Sending Accept header '*/*' because unexpected returned format 'json' in a 'DESCRIBE' SPARQL query form
  warnings.warn(


In [21]:
response.convert(JSON)

NameError: name 'JSON' is not defined

In [ ]:
acervoQuery.endpoint_url('')

Title

In [3]:
acervoUpdate = FusekiUpdate('http://localhost:3030', 'acervo')
work_id = '2'

up = """
PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

WITH <https://bibliokeia.com/resources/work/bk-"""+work_id+""">
DELETE
{
   <https://bibliokeia.com/resources/work/bk-"""+work_id+"""> bf:title ?o .
  	?o ?p ?title .
}
INSERT
{
   <https://bibliokeia.com/resources/work/bk-"""+work_id+"""> bf:title ?o .
  	?o rdfs:label "TESTE" .

    <https://bibliokeia.com/resources/work/bk-"""+work_id+"""> bf:title ?o .
  	?o bf:mainTitle "TESTE" .
}
WHERE { 
<https://bibliokeia.com/resources/work/bk-"""+work_id+"""> bf:title ?o .
  	?o ?p ?title .
    }
"""

response = acervoUpdate.run_sparql(up)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [4]:
obj = {
    "title": "TESTE",
    "typeLabel": "Text",
    "contribution": "Popper, Karl R. (Karl Raimund), 1902-1994",
    "serie": "Pensamento científico",
    "hasInstance": "https://bibliokeia.com/resources/instance/bk-3",
    "instanceID": "bk-3",
    "contentType": "Text",
    "mainTitle": "TESTE",
    "subtitle": "",
    "contributionAgent": "Popper, Karl R. (Karl Raimund), 1902-1994",
    "contributionRole": "Autor",
    "contributionRoleUri": "https://bibliokeia.com/authorities/names/n80032184",
    "contributionID": "n80032184",
    "subjects": [
        {
            "uri": "https://bibliokeia.com/authorities/subjects/sh85084414",
            "label": "Metodologia"
        },
        {
            "uri": "https://bibliokeia.com/authorities/subjects/sh85118553",
            "label": "Ciência"
        }
    ],
    "language": "",
    "languageCode": "",
    "cdd": "548.2",
    "cutter": "A587s",
    "serieURI": "https://bibliokeia.com/resources/hub/bk-1"
}

In [5]:
from api.src.schemas.bibframe.work import Work_Schema

In [8]:
work = Work_Schema(**obj)

In [53]:
work.contentType

'Text'

In [22]:
from SPARQLWrapper import SPARQLWrapper, JSON

In [47]:
sparql = SPARQLWrapper('http://localhost:3030/acervo/sparql')
sparql.setReturnFormat(JSON)

In [58]:
sparql_str = """
    PREFIX work: <https://bibliokeia.com/resources/work/>
PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
SELECT ?o 
WHERE { 
  graph work:bk-2
  {work:bk-2 rdf:type ?o}}
"""
sparql.setQuery(sparql_str)
ret = sparql.queryAndConvert()
bindings = ret['results']['bindings']
bindings

[{'o': {'type': 'uri', 'value': 'http://id.loc.gov/ontologies/bibframe/Work'}},
 {'o': {'type': 'uri', 'value': 'http://id.loc.gov/ontologies/bibframe/Text'}}]

In [64]:
for i in bindings:
    content = i['o']['value'].split('/')[-1]
    if content != 'Work':
        print(content)

Text


In [63]:
w = Work_Schema(contentType='Work')


ValidationError: 10 validation errors for Work_Schema
mainTitle
  field required (type=value_error.missing)
contributionAgent
  field required (type=value_error.missing)
contributionID
  field required (type=value_error.missing)
contributionRole
  field required (type=value_error.missing)
contributionRoleUri
  field required (type=value_error.missing)
subjects
  field required (type=value_error.missing)
language
  field required (type=value_error.missing)
languageCode
  field required (type=value_error.missing)
cdd
  field required (type=value_error.missing)
cutter
  field required (type=value_error.missing)

In [36]:
for s, p, o  in ret:
    print(s, p, o)

https://bibliokeia.com/resources/work/bk-2 http://id.loc.gov/ontologies/bibframe/classification N3d389a07c681487cb1a0a27ed36d8065
https://bibliokeia.com/resources/work/bk-2 http://id.loc.gov/ontologies/bibframe/language http://id.loc.gov/vocabulary/languages/por
N8f0b10f10bf54c4bb3dc94d348a63537 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://id.loc.gov/ontologies/bibframe/GenerationProcess
Ne09edaa8c31345778b0c310a320702f9 http://www.w3.org/2000/01/rdf-schema#label Popper, Karl R. (Karl Raimund), 1902-1994
N778258d5644147c08a33671553794737 http://id.loc.gov/ontologies/bibframe/mainTitle TESTE
https://bibliokeia.com/resources/work/bk-2 http://id.loc.gov/ontologies/bibframe/contribution Ne09edaa8c31345778b0c310a320702f9
https://bibliokeia.com/resources/work/bk-2 http://id.loc.gov/ontologies/bibframe/hasSeries https://bibliokeia.com/resources/hub/bk-1
N021e7b7a4a414f6592d66c02ce4376ff http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://id.loc.gov/ontologies/bibframe/AdminMetada